# pandas的拼接操作

pandas的拼接分为两种：

- 级联：pd.concat, pd.append (没有重复数据)
- 合并：pd.merge, pd.join (有重复数据)

## 0. 回顾numpy的级联

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

In [60]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [66]:
nd1 = np.random.randint(0,10,size=(3,3))
nd2 = np.random.randint(10,100,size=(3,3))
display(nd1,nd2)
np.concatenate((nd1,nd2))
np.concatenate((nd1,nd2),axis=1)

array([[7, 4, 2],
       [2, 7, 9],
       [7, 6, 6]])

array([[69, 93, 71],
       [87, 25, 38],
       [42, 15, 35]])

array([[ 7,  4,  2, 69, 93, 71],
       [ 2,  7,  9, 87, 25, 38],
       [ 7,  6,  6, 42, 15, 35]])

## 1. 使用pd.concat()级联

为方便讲解，我们首先定义一个生成DataFrame的函数：

In [68]:
df1 = DataFrame(nd1)
df2 = DataFrame(nd2)
display(df1,df2)

,0,1,2
0,7,4,2
1,2,7,9
2,7,6,6


,0,1,2
0,69,93,71
1,87,25,38
2,42,15,35


### 1)  简单级联

pandas使用pd.concat函数，与np.concatenate函数类似

In [72]:
df3 = pd.concat((df1,df2)) #默认 axis是0 是纵向拼接
df3

,0,1,2
0,7,4,2
1,2,7,9
2,7,6,6
0,69,93,71
1,87,25,38
2,42,15,35


In [73]:
# 索引有重复 会产生一些问题
df3.loc[0]

,0,1,2
0,7,4,2
0,69,93,71


In [81]:
# 可以通过 重置索引的方式 去重新让索引不重复
# ignore_index=False 忽略原索引 建立新索引 默认是False
df3 = pd.concat((df1,df2),ignore_index=True)
df3

,0,1,2
0,7,4,2
1,2,7,9
2,7,6,6
3,69,93,71
4,87,25,38
5,42,15,35


In [87]:
display(df1,df2)

,0,1,2
0,7,4,2
1,2,7,9
2,7,6,6


,0,1,2
0,69,93,71
1,87,25,38
2,42,15,35


In [88]:
df3 = pd.concat((df1,df2),ignore_index=True,axis=0) # 默认是 0 竖直方向
df3 = pd.concat((df1,df2),axis=1) # 1 水平方向拼接
df3 = pd.concat((df1,df2),ignore_index=True,axis=1)
df3

,0,1,2,3,4,5
0,7,4,2,69,93,71
1,2,7,9,87,25,38
2,7,6,6,42,15,35


In [89]:
display(df1,df2)

,0,1,2
0,7,4,2
1,2,7,9
2,7,6,6


,0,1,2
0,69,93,71
1,87,25,38
2,42,15,35


In [91]:
df3 = pd.concat((df1,df2),keys=["第一个","第二个"])
df3

0   1   2
第一个 0   7   4   2
    1   2   7   9
    2   7   6   6
第二个 0  69  93  71
    1  87  25  38
    2  42  15  35

### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有3种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

- 连接指定轴 join_axes

In [93]:
nd1,nd2

(array([[7, 4, 2],
        [2, 7, 9],
        [7, 6, 6]]), array([[69, 93, 71],
        [87, 25, 38],
        [42, 15, 35]]))

In [99]:
# 把相同索引的行或列进行级联，如果存在不匹配的行列标签，补nan
df3 = DataFrame(data=nd1,columns=list("ABC"))
df4 = DataFrame(data=nd2,columns=list("BCD"))
display(df3,df4)

,A,B,C
0,7,4,2
1,2,7,9
2,7,6,6


,B,C,D
0,69,93,71
1,87,25,38
2,42,15,35


In [98]:
pd.concat((df3,df4),sort=False) # dataframe 拼接 默认是 外联 

,A,B,C,D
0,7.0,4,2,NaN
1,2.0,7,9,NaN
2,7.0,6,6,NaN
0,NaN,69,93,71.0
1,NaN,87,25,38.0
2,NaN,42,15,35.0


In [101]:
pd.concat((df3,df4),sort=False,) #{'inner', 'outer'}, default 'outer'
pd.concat((df3,df4),sort=False,join="inner") 

,B,C
0,4,2
1,7,9
2,6,6
0,69,93
1,87,25
2,42,15


In [36]:
pd.concat((df3,df4),sort="False",axis=1,ignore_index="True")

,0,1,2,3,4,5
0,8,1,7,48,57,94
1,9,2,3,96,73,20
2,7,0,9,93,17,13


In [42]:
# 使用keys参数，可以自动设置为多层级索引，避免索引重复
pd.concat((df3,df4),sort="False")
pd.concat((df3,df4),keys=['期中','期末'],sort="False")

A   B   C     D
期中 0  8.0   1   7   NaN
   1  9.0   2   3   NaN
   2  7.0   0   9   NaN
期末 0  NaN  48  57  94.0
   1  NaN  96  73  20.0
   2  NaN  93  17  13.0

In [45]:
pd.concat((df3,df4),sort="False",join="inner")

,B,C
0,1,7
1,2,3
2,0,9
0,48,57
1,96,73
2,93,17


In [103]:
df3

,A,B,C
0,7,4,2
1,2,7,9
2,7,6,6


In [104]:
df4

,B,C,D
0,69,93,71
1,87,25,38
2,42,15,35


In [107]:
index = pd.Index(["B","C"])
pd.concat((df3,df4),sort="False",join_axes=[index])



,B,C
0,4,2
1,7,9
2,6,6
0,69,93
1,87,25
2,42,15


In [ ]:
pd.concat() 
# 参数
#1. objs 传入列表或者元素 里面是要拼接的DataFrame
#2. axis 拼接的时候是沿着什么方向 默认值是0 纵向 如果是1就是横向
#3. join 指定了拼接的方式 默认是outer
#  outer 外联 所有的列都会拼进来
#  inner 内联 只有那些两个DataFrame都有的列才会拼进来
#4. join_axes 直接指定那些列要放进来
#5. ignore_index=False 忽略原有索引创建新的索引 （如果索引有重复可以通过忽略原索引来重置）
# 6. keys 可以把不同的DataFrame分成多组 也可以用来解决index重复的问题

In [48]:
pd.concat((df3,df4),sort="False",axis=1)

,A,B,C,B,C,D
0,8,1,7,48,57,94
1,9,2,3,96,73,20
2,7,0,9,93,17,13


In [54]:
axes = pd.Index(["B","C"])
pd.concat((df3,df4),sort="False",join_axes=[axes])

,B,C
0,1,7
1,2,3
2,0,9
0,48,57
1,96,73
2,93,17


============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表df

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

In [113]:
import numpy as np

data = np.random.randint(0,150,size=(2,3))
index = ["张三","李四"]
columns = ["语文","数学","外语"]
df1 = DataFrame(data,index,columns)
df1

,语文,数学,外语
张三,133,34,105
李四,10,20,114


In [112]:

data = np.random.randint(0,150,size=(2,1))
index = ["张三","李四"]
columns = ["计算机"]
df2 = DataFrame(data,index,columns)
df2

,计算机
张三,102
李四,43


In [127]:
df5 = pd.concat((df1,df2),axis=1,sort=False)
df5

,语文,数学,外语,计算机
张三,133,34,105,102
李四,10,20,114,43


In [118]:
data = np.random.randint(0,150,size=(1,4))
index = ["王老五"]
columns = ["语文","数学","外语","计算机"]
df3 = DataFrame(data,index,columns)
df3

,语文,数学,外语,计算机
王老五,144,117,9,92


In [121]:
pd.concat((df1,df3),sort=False)

,语文,数学,外语,计算机
张三,133,34,105,NaN
李四,10,20,114,NaN
王老五,144,117,9,92.0


### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [130]:
df5

,语文,数学,外语,计算机
张三,133,34,105,102
李四,10,20,114,43


In [126]:
df3

,语文,数学,外语,计算机
王老五,144,117,9,92


In [140]:
df6= df5.append(df3)
df6

,语文,数学,外语,计算机
张三,133,34,105,102
李四,10,20,114,43
王老五,144,117,9,92


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

In [155]:
data = np.random.randint(0,150,size=(3,4))
index = ["张三","李四","王老五"]
columns = ["语文","数学","外语","计算机"]
df7 = DataFrame(data,index,columns)
display(df6,df7)

,语文,数学,外语,计算机
张三,133,34,105,102
李四,10,20,114,43
王老五,144,117,9,92


,语文,数学,外语,计算机
张三,30,17,137,59
李四,130,74,110,97
王老五,50,24,38,8


In [157]:
df6.append(df7) # 只能是把行 纵向地 从后 往前 拼接

,语文,数学,外语,计算机
张三,133,34,105,102
李四,10,20,114,43
王老五,144,117,9,92
张三,30,17,137,59
李四,130,74,110,97
王老五,50,24,38,8


In [160]:
pd.concat((df6,df7),axis=1,keys=["期中","期末"])

期中                  期末             
      语文   数学   外语  计算机   语文  数学   外语 计算机
张三   133   34  105  102   30  17  137  59
李四    10   20  114   43  130  74  110  97
王老五  144  117    9   92   50  24   38   8

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

###  1) 一对一合并

### 2) 多对一合并

### 3) 多对多合并

In [317]:
# 同学们使用这个方法可能会报错 报错会说 缺少一个库 大家通过pip安装一下就可以了
table1 = pd.read_excel("./关系表.xls",sheet_name=0)
table2 = pd.read_excel("./关系表.xls",sheet_name=1)
table3 = pd.read_excel("./关系表.xls",sheet_name=2)
table4 = pd.read_excel("./关系表.xls",sheet_name=3)
table5 = pd.read_excel("./关系表.xls",sheet_name=4)

In [289]:
display(table1,table2)

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [290]:
# 一一对应的表格 通过merge融合 把数据对应上就可以了
pd.merge(table1,table2)
# how指的是如何拼接 默认是inner 内联 （两个表格都有的项目才留下）

,手机型号,参考价格,重量
0,windowsPhone,2500,0.50
1,iPhone,7500,0.40
2,Android,4000,0.45


In [294]:
# {'left', 'right', 'outer', 'inner'}, default 'inner' 
pd.merge(table1,table2)
pd.merge(table1,table2,how="inner") # inner  是取交集 两个都有的项目才出现
pd.merge(table1,table2,how="outer") # outer 是取并集 任何一个表格里出现的项目都会出现
pd.merge(table1,table2,how="left") # 左边的表格有多少项目 这里就有多少项目
pd.merge(table1,table2,how="right") # 右边的表格有多少项目 这里就有多少项目

,手机型号,参考价格,重量
0,windowsPhone,2500,0.50
1,iPhone,7500,0.40
2,Android,4000,0.45


In [300]:
# 一对多
display(table2,table3)

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


,经销商,发货地区,手机型号
0,pegge,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


In [305]:
# 一对多 的 表格拼接 两个表格需要有一个相同的column 然后在把1*多个 计算出新的行
pd.merge(table2,table3,how="inner")
pd.merge(table2,table3,how="outer")
pd.merge(table2,table3,how="left")
pd.merge(table2,table3,how="right")

,手机型号,重量,经销商,发货地区
0,windowsPhone,0.50,petter,shenzhen
1,iPhone,0.40,pegge,beijing
2,iPhone,0.40,tom,guangzhou
3,Android,0.45,lucy,beijing
4,Android,0.45,mery,guangzhou


In [306]:
# 多对多
display(table3,table4)

,经销商,发货地区,手机型号
0,pegge,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


In [311]:
# 多对多 拼接
# 方式1 我们通过on 指定 按照哪一列进行拼接 然后可以通过suffixes指定重复的列的后缀
pd.merge(table3,table4,on="手机型号",suffixes=["_1","_2"])

,经销商,发货地区_1,手机型号,发货地区_2,价格
0,pegge,beijing,iPhone,beijing,7000
1,pegge,beijing,iPhone,guangzhou,7600
2,pegge,beijing,iPhone,shenzhen,7400
3,tom,guangzhou,iPhone,beijing,7000
4,tom,guangzhou,iPhone,guangzhou,7600
5,tom,guangzhou,iPhone,shenzhen,7400
6,lucy,beijing,Android,beijing,3600
7,lucy,beijing,Android,guangzhou,4200
8,lucy,beijing,Android,shenzhen,3900
9,mery,guangzhou,Android,beijing,3600


In [313]:
# 第二种方式 指定两个相同的列 这两列中的项目必须都对应上才会显示到新的表格中
pd.merge(table3,table4,on=["手机型号","发货地区"])

,经销商,发货地区,手机型号,价格
0,pegge,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


,经销商,发货地区,手机型号
0,pegge,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


,经销商,发货地区_1,手机型号,发货地区_2,价格
0,pegge,beijing,iPhone,beijing,7000
1,pegge,beijing,iPhone,guangzhou,7600
2,pegge,beijing,iPhone,shenzhen,7400
3,tom,guangzhou,iPhone,beijing,7000
4,tom,guangzhou,iPhone,guangzhou,7600
5,tom,guangzhou,iPhone,shenzhen,7400
6,lucy,beijing,Android,beijing,3600
7,lucy,beijing,Android,guangzhou,4200
8,lucy,beijing,Android,shenzhen,3900
9,mery,guangzhou,Android,beijing,3600


,经销商,发货地区,手机型号,价格
0,pegge,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


,发货地区,型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


,手机型号,参考价格,发货地区,价格
0,windowsPhone,2500,beijing,2300
1,windowsPhone,2500,guangzhou,2800
2,windowsPhone,2500,shenzhen,2750
3,iPhone,7500,beijing,7000
4,iPhone,7500,guangzhou,7600
5,iPhone,7500,shenzhen,7400
6,Android,4000,beijing,3600
7,Android,4000,guangzhou,4200
8,Android,4000,shenzhen,3900


In [319]:
display(table4,table5)

,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


,型号,价格
0,iPhone,7000
1,windowsPhone,2300
2,Android,3600
3,iPhone,7600
4,windowsPhone,2800
5,Android,4200
6,iPhone,7400
7,windowsPhone,2750
8,Android,3900


In [318]:
pd.merge(table4,table5)

,发货地区,手机型号,价格,型号
0,beijing,iPhone,7000,iPhone
1,beijing,windowsPhone,2300,windowsPhone
2,beijing,Android,3600,Android
3,guangzhou,iPhone,7600,iPhone
4,guangzhou,windowsPhone,2800,windowsPhone
5,guangzhou,Android,4200,Android
6,shenzhen,iPhone,7400,iPhone
7,shenzhen,windowsPhone,2750,windowsPhone
8,shenzhen,Android,3900,Android


### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

- 外合并 how='outer'：补NaN

- 左合并、右合并：how='left'，how='right'，

============================================

练习17：



1. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

作业知识补充

In [321]:
s1 = Series(["A","B","C","B"])
s1

0    A
1    B
2    C
3    B
dtype: object

In [322]:
s1.unique()

array(['A', 'B', 'C'], dtype=object)

首先导入文件，并查看数据样本

In [325]:
df_abbr = pd.read_csv("../data/state-abbrevs.csv") # csv文件的数据导入后 会变成DataFrame供我们使用
df_abbr.shape

(51, 2)

In [326]:
df_areas = pd.read_csv("../data/state-areas.csv") # csv文件的数据导入后 会变成DataFrame供我们使用
df_areas

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [329]:
df_pop = pd.read_csv("../data/state-population.csv") # csv文件的数据导入后 会变成DataFrame供我们使用
df_pop

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
5,AL,total,2011,4801627.0
6,AL,total,2009,4757938.0
7,AL,under18,2009,1134192.0
8,AL,under18,2013,1111481.0
9,AL,total,2013,4833722.0


合并popu与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

In [333]:
df_abbr

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [332]:
df_pop

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
5,AL,total,2011,4801627.0
6,AL,total,2009,4757938.0
7,AL,under18,2009,1134192.0
8,AL,under18,2013,1111481.0
9,AL,total,2013,4833722.0


In [339]:

df_pop2 = pd.merge(df_abbr,df_pop,left_on="abbreviation",right_on="state/region",how="outer")
df_pop3 = df_pop2.drop(labels="abbreviation",axis=1)

去除abbreviation的那一列（axis=1）

In [342]:
df_pop3

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0
5,Alabama,AL,total,2011,4801627.0
6,Alabama,AL,total,2009,4757938.0
7,Alabama,AL,under18,2009,1134192.0
8,Alabama,AL,under18,2013,1111481.0
9,Alabama,AL,total,2013,4833722.0


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [343]:
df_pop3.isnull().any() # isnull()有空值是True 没有空值是False any()只要有True就是True
# 合在一起使用 就是 这一列中 只要有空值就是true
# 接下来我们就要对空值进行处理了


state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

In [349]:
df_pop3["state"].isnull() #这是一个序列 有值是False 没有值是True
df_pop3[df_pop3["state"].isnull()] #DataFrame 后面 的中括号 中可以传入 序列 如果序列中是布尔值 False这这一项不取 True就取出这一项
df_pop3[df_pop3["state"].isnull()] # 取出来的都是有空值的
df_pop3[df_pop3["state"].isnull()]["state/region"].unique()


array(['PR', 'USA'], dtype=object)

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

In [393]:
# df_pop3[df_pop3["state"].isnull()]["state"] = "Puerto Rico" # 为了安全不能直接设置值
temp = df_pop3[df_pop3["state"].isnull()].copy()
temp["state"] = "Puerto Rico"
temp
temp2 = df_pop3[df_pop3["state/region"]=="USA"].copy()
temp2["state"]="United States"
temp2
df_pop3[df_pop3["state/region"]=="USA"] = temp2

In [394]:
df_pop3[df_pop3["state"].isnull()] = temp # 不能直接操作值 但是可以把DataFrame赋值给DataFrame
df_pop3.isnull().any()
df_pop4 = df_pop3.dropna()

In [395]:
df_pop4.isnull().any()

state           False
state/region    False
ages            False
year            False
population      False
dtype: bool

合并各州人口数据和面积数据areas，使用外合并。

思考一下为什么使用外合并？


In [417]:
df_pop_area = pd.merge(df_pop4,df_areas,how="outer")
df_pop_area.dropna()

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0
5,Alabama,AL,total,2011,4801627.0,52423.0
6,Alabama,AL,total,2009,4757938.0,52423.0
7,Alabama,AL,under18,2009,1134192.0,52423.0
8,Alabama,AL,under18,2013,1111481.0,52423.0
9,Alabama,AL,total,2013,4833722.0,52423.0


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
5,AL,total,2011,4801627.0,Alabama
6,AL,total,2009,4757938.0,Alabama
7,AL,under18,2009,1134192.0,Alabama
8,AL,under18,2013,1111481.0,Alabama
9,AL,total,2013,4833722.0,Alabama


In [376]:
df_pop_area

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423
1,Alabama,AL,total,2012,4817528.0,52423
2,Alabama,AL,under18,2010,1130966.0,52423
3,Alabama,AL,total,2010,4785570.0,52423
4,Alabama,AL,under18,2011,1125763.0,52423
5,Alabama,AL,total,2011,4801627.0,52423
6,Alabama,AL,total,2009,4757938.0,52423
7,Alabama,AL,under18,2009,1134192.0,52423
8,Alabama,AL,under18,2013,1111481.0,52423
9,Alabama,AL,total,2013,4833722.0,52423


找出2010年的全民人口数据,df.query(查询语句)

In [418]:
df_2010 = df_pop_area.query("year==2010 & ages=='total'")
df_2010.shape
df_2010.dropna()

,state,state/region,ages,year,population,area (sq. mi)
3,Alabama,AL,total,2010,4785570.0,52423.0
91,Alaska,AK,total,2010,713868.0,656425.0
101,Arizona,AZ,total,2010,6408790.0,114006.0
189,Arkansas,AR,total,2010,2922280.0,53182.0
197,California,CA,total,2010,37333601.0,163707.0
283,Colorado,CO,total,2010,5048196.0,104100.0
293,Connecticut,CT,total,2010,3579210.0,5544.0
379,Delaware,DE,total,2010,899711.0,1954.0
389,District of Columbia,DC,total,2010,605125.0,68.0
475,Florida,FL,total,2010,18846054.0,65758.0


对查询结果进行处理，以state列作为新的行索引:set_index

In [426]:
df_2010.set_index("state")

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0
Colorado,CO,total,2010,5048196.0,104100.0
Connecticut,CT,total,2010,3579210.0,5544.0
Delaware,DE,total,2010,899711.0,1954.0
District of Columbia,DC,total,2010,605125.0,68.0


计算人口密度。注意是Series/Series，其结果还是一个Series。

In [430]:
df_2010 = df_2010.dropna()

In [416]:
df_2010["area (sq. mi)"]

3        52423.0
91      656425.0
101     114006.0
189      53182.0
197     163707.0
283     104100.0
293       5544.0
379       1954.0
389         68.0
475      65758.0
485      59441.0
570      10932.0
581      83574.0
666      57918.0
677      36420.0
762      56276.0
773      82282.0
858      40411.0
869      51843.0
954      35387.0
965     147046.0
1050     77358.0
1061    110567.0
1146      9351.0
1157      8722.0
1242    121593.0
1253     54475.0
1338     53821.0
1349     70704.0
1434     44828.0
1445     69903.0
1530     98386.0
1541     12407.0
1626     10555.0
1637     96810.0
1722     86943.0
1733     48434.0
1818     69709.0
1829     46058.0
1914      1545.0
1925     32007.0
2010     77121.0
2021     42146.0
2106    268601.0
2117     84904.0
2202      9615.0
2213     42769.0
2298     71303.0
2309     24231.0
2394     65503.0
2405     97818.0
2470      3515.0
2519         NaN
Name: area (sq. mi), dtype: float64

In [431]:
dens = df_2010["population"]/df_2010["area (sq. mi)"]
dens

3         91.287603
91         1.087509
101       56.214497
189       54.948667
197      228.051342
283       48.493718
293      645.600649
379      460.445752
389     8898.897059
475      286.597129
485      163.409902
570      124.746707
581       18.794338
666      221.687472
677      178.197831
762       54.202751
773       34.745266
858      107.586994
869       87.676099
954       37.509990
965        6.736171
1050      23.654153
1061      24.448796
1146     140.799273
1157    1009.253268
1242      16.982737
1253     356.094135
1338     177.617157
1349       9.537565
1434     257.549634
1445      53.778278
1530      39.001565
1541     466.445797
1626     621.815538
1637     102.015794
1722      61.078373
1733      61.321530
1818      86.015622
1829     275.966651
1914     681.339159
1925     144.854594
2010      10.583512
2021     150.825298
2106      93.987655
2117      32.677188
2202      65.085075
2213     187.622273
2298      94.557817
2309      76.519582
2394      86.851900


排序，并找出人口密度最高的五个州sort_values()的密度

In [434]:
type(dens.sort_values().tail(6))
# dens.sort_values().tail(6).iloc[:-1]
dens.sort_values().tail(5)

293      645.600649
1914     681.339159
1157    1009.253268
2470    1058.665149
389     8898.897059
dtype: float64

找出人口密度最低的五个州的密度

In [421]:
dens.sort_values().head()

91       1.087509
2405     5.768079
965      6.736171
1349     9.537565
2010    10.583512
dtype: float64

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息